In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_percentage_error, median_absolute_error
from datetime import datetime
import json
import os

from garisom_opt import GarisomModel
from garisom_opt.metric import normalized_nash_sutcliffe_efficiency

In [ ]:
pop = 1 # ccr/jla/tsz/nrv

In [ ]:
def get_output_from_result(result_dir: str, params, model_dir, config_file):
    results_file = os.path.join(result_dir, "results.json")
    with open(results_file, "r") as f:
        results = json.load(f)
    
    optim_params = results['parameters']

    results = {}

    for param, set in optim_params.items():
        out = GarisomModel.run(
            X=set,
            params=params,
            population=pop,
            config_file=config_file,
            model_dir=model_dir,
        )
        results[param] = out

    return results

In [ ]:
params = pd.read_csv("./DBG/parameters.csv")
model_dir = './garisom/02_program_code'
config_file = os.path.abspath("./DBG/configuration.csv")
result_dir=None

In [ ]:
# result_dir = "optimization/output/ccr/pressures/20250709_143431"
# choice = 'P-MD'

result_dir = "optimization/output/ccr/leaftemp/20250709_143426"
choice = 'leaftemp.b'

In [ ]:
match pop:
    case 1:
        ground = pd.read_csv("data/ccr_hourly_data.csv")
        stderr = pd.read_csv("data/ccr_std_error.csv")
        # pred = pd.read_csv("./garisom/02_program_code/out/timesteps_output_POFR_CCR-COL_DBG.csv").iloc[:-1]
        pred = pd.read_csv("./garisom/02_program_code/timesteps_output_POFR_CCR-COL_DBG.csv").iloc[:-1]
        low = pd.read_csv("./garisom/02_program_code/low/timesteps_output_POFR_CCR-COL_DBG.csv").iloc[:-1]
        high = pd.read_csv("./garisom/02_program_code/high/timesteps_output_POFR_CCR-COL_DBG.csv").iloc[:-1]
    case 2:
        ground = pd.read_csv("data/jla_hourly_data.csv")
        stderr = pd.read_csv("data/jla_std_error.csv")
        pred = pd.read_csv("./garisom/02_program_code/out/timesteps_output_POFR_JLA-JAK_DBG.csv").iloc[:-1]
    case 3:
        ground = pd.read_csv("data/tsz_hourly_data.csv")
        stderr = pd.read_csv("data/tsz_std_error.csv")
        pred = pd.read_csv("./garisom/02_program_code/out/timesteps_output_POFR_TSZ-SAN_DBG.csv").iloc[:-1]
    case 4:
        ground = pd.read_csv("data/nrv_hourly_data.csv")
        stderr = pd.read_csv("data/nrv_std_error.csv")
        pred = pd.read_csv("./garisom/02_program_code/out/timesteps_output_POFR_NRV-NEW_DBG.csv").iloc[:-1]

In [ ]:
if result_dir is not None:
    pred_choices = get_output_from_result(
        result_dir,
        params=params,
        model_dir=model_dir,
        config_file=config_file
    )
    pred = pred_choices[choice].iloc[:-1]

In [ ]:
ground.columns

In [ ]:
pred.columns

In [ ]:
def cmp_pred_to_ground_metrics(columns, ground, pred, start, end):

    fit = {}

    for col in columns:

        col_ground = ground[ground['julian-day'].between(start, end)][col].dropna()
        col_pred = pred[col]

        col_pred = col_pred.loc[col_ground.index]
        
        mse = mean_squared_error(col_ground, col_pred)
        rmse = root_mean_squared_error(col_ground, col_pred)
        mape = mean_absolute_percentage_error(col_ground, col_pred)
        made = median_absolute_error(col_ground, col_pred)
        r2 = r2_score(col_ground, col_pred)
        nnse = normalized_nash_sutcliffe_efficiency(col_ground, col_pred)

        fit[col] = {
            'mse' : mse,
            'rmse' : rmse,
            'mape' : mape,
            'made' : made,
            'r2' : r2,
            'nse' : nnse
        }

        # Plot ground vs pred with fitted line and 1:1 correspondence
        plt.figure(figsize=(8, 6))
        plt.scatter(col_ground, col_pred, label='Data Points', alpha=0.7)
        plt.plot([col_ground.min(), col_ground.max()], [col_ground.min(), col_ground.max()], 'r--', label='1:1 Line')
        
        # Fit a line to the data
        fit_coeff = np.polyfit(col_ground, col_pred, 1)
        fit_line = np.poly1d(fit_coeff)
        plt.plot(col_ground, fit_line(col_ground), 'b-', label=f'Fitted Line: y={fit_coeff[0]:.2f}x+{fit_coeff[1]:.2f}')
        
        # Add metrics as text
        plt.text(0.05, 0.95, ''.join([f'{key}: {value:.2f}\n' for key, value in fit[col].items()]), 
             transform=plt.gca().transAxes, fontsize=10, verticalalignment='top')
        
        plt.xlabel('Ground')
        plt.ylabel('Prediction')
        plt.title(f'Ground vs Prediction: {col}')
        plt.legend()
        plt.grid(True)
        plt.show()

    return fit

In [ ]:
columns = ['P-PD', 'P-MD', 'GW', 'K-plant', 'E-MD', 'leaftemp']
start_day = 201
stress_begin = 223
predrought_cutoff = 236
drought = 240           # drought measurement period, collected 3 days after initial drought began
post_drought = 241
leaft_data_start = 205
end_day = 266

In [ ]:
start_timestep = pred[pred['julian-day'] == start_day].index[0]
predrought_timestep = pred[pred['julian-day'] == predrought_cutoff].index[0]
drought_timestep = pred[pred['julian-day'] == drought].index[0]
post_timestep = pred[pred['julian-day'] == post_drought].index[0]
stress_timestep = pred[pred['julian-day'] == stress_begin].index[0]
leaft_start_timestep = pred[pred['julian-day'] == leaft_data_start].index[0]

print(f"Start Timestep: {start_timestep}, Predrought Timestep: {predrought_timestep}, Drought Timestep: {drought_timestep}, Leaftemp Start: {leaft_start_timestep}")

In [ ]:
res = cmp_pred_to_ground_metrics(columns, ground, pred, start_day, predrought_cutoff)

In [ ]:
res

In [ ]:
unit = {
    "P-PD" : "-MPa",
    "P-MD" : "-MPa",
    "GW" : "mmol s-1 m-2 (LA)",
    "E-MD" : "mmol s-1 m-2 (LA)",
    "K-plant" : "kg hr-1 m-2 (BA)",
    'leaftemp' : "C"
}

In [ ]:
def plot_ground_pred(stop, start=0):

    ground_pre_and_drought = ground[start:stop].copy(deep=True)
    pred_pre_and_drought = pred[start:stop].copy(deep=True)
    pre_and_drough_low = low[start:stop].copy(deep=True)
    pre_and_drought_high = high[start:stop].copy(deep=True)
    pre_and_drough_low.loc[pre_and_drough_low['GW'] > 1000, 'GW'] = 0
    pre_and_drought_high.loc[pre_and_drought_high['GW'] > 1000, 'GW'] = 0
    pred_pre_and_drought.loc[pred_pre_and_drought['GW'] > 1000, 'GW'] = 0

    for col in columns:

        # Make plot
        plt.figure(figsize=(10, 6))
        plt.scatter(range(start, stop), ground_pre_and_drought[col], label=f"Ground", color='navy')
        plt.plot(range(start, stop), pred_pre_and_drought[col], color="green", alpha=1, label=f'Prediction')
        # plt.fill_between(range(start, stop), pre_and_drough_low[col], pre_and_drought_high[col], color="green", alpha=0.4, label=f'Prediction 95%')
        plt.ylim(bottom=0)
        plt.title(f"Pred versus Ground: {col}")
        plt.xlabel("Timestep")
        plt.ylabel(f"{col} {unit[col]}")
        plt.axvline(x=stress_begin, color='navy', linestyle='--', label='Pre-stress Cutoff')
        plt.axvline(x=predrought_timestep, color='navy', linestyle='--', label='Predrought Cutoff')
        plt.axvline(x=post_timestep, color='navy', linestyle='--', label='Drought Cutoff')
        plt.legend()

    plt.figure(figsize=(10, 6))
    plt.plot(range(start, stop), pred_pre_and_drought['end-PLC-plant'], color="green", alpha=1, label=f'Prediction')
    plt.title(f"Predicted PLC")
    plt.xlabel("Timestep")
    plt.ylabel("PLC %")
    plt.axvline(x=stress_begin, color='navy', linestyle='--', label='Pre-stress Cutoff')
    plt.axvline(x=predrought_timestep, color='navy', linestyle='--', label='Predrought Cutoff')
    plt.axvline(x=post_timestep, color='navy', linestyle='--', label='Drought Cutoff')
    plt.legend()

    # Compute cumulative PLC
    cumulative_plc = pred_pre_and_drought['end-PLC-plant'].cumsum()
    plt.figure(figsize=(10, 6))
    plt.plot(range(start, stop), cumulative_plc, color="blue", alpha=1, label=f'Cumulative PLC')
    plt.title(f"Cumulative PLC")
    plt.xlabel("Timestep")
    plt.ylabel("Cumulative PLC %")
    plt.axvline(x=stress_begin, color='navy', linestyle='--', label='Pre-stress Cutoff')
    plt.axvline(x=predrought_timestep, color='navy', linestyle='--', label='Predrought Cutoff')
    plt.axvline(x=post_timestep, color='navy', linestyle='--', label='Drought Cutoff')
    plt.legend()

    # Summary stats
    print("Ground")
    display(ground_pre_and_drought[columns].describe())
    print("Pred")
    display(pred_pre_and_drought[columns].describe())

In [ ]:
plot_ground_pred(predrought_timestep)

In [ ]:
res = cmp_pred_to_ground_metrics(columns, ground, pred, drought, end_day)

In [ ]:
res

In [ ]:
res = cmp_pred_to_ground_metrics(columns, ground, pred, start_day, end_day)

In [ ]:
res

In [ ]:
plot_ground_pred(len(pred))

In [ ]:
def plot_diurnal(var, day, hours, xerr):
    plt.figure(figsize=(10, 6))
    plt.plot(range(0, 24), pred.loc[pred['julian-day'] == day][var], label='Prediction', color='green')
    # plt.fill_between(range(0, 24), low.loc[low['julian-day'] == day][var], pred.loc[pred['julian-day'] == day][var], color='green', alpha=0.4)
    # plt.fill_between(range(0, 24), pred.loc[pred['julian-day'] == day][var], high.loc[pred['julian-day'] == day][var], color='green', alpha=0.4)
    plt.scatter(hours, ground.loc[(ground['julian-day'] == day) & (ground['standard-time'].isin(hours))][var], label='Ground Truth', color='navy')

    for hour in hours:
        try:
            m_stderr = stderr.loc[(stderr['julian-day'] == day) & (stderr['standard-time'] == hour)][var].item()
            m_ground = ground.loc[(ground['julian-day'] == day) & (ground['standard-time'] == hour)][var].item()
            plt.errorbar(hour, m_ground, yerr=1.96*m_stderr, xerr=xerr, color='navy')
        except:
            pass

    plt.xticks(range(0, 24))
    plt.xlim(0, 23)  # Ensure no gap on the left side of the origin
    plt.ylim(0, np.minimum(1000, np.max(pred[var]) + 0.1 * np.max(pred[var])))
    plt.xlabel('Hour of Day')
    plt.ylabel(var)
    plt.title(f'Diurnal Plot of {var} on Julian Day {day}')
    plt.legend()
    plt.grid(False)
    plt.show()

In [ ]:
day=236
plot_diurnal('P-PD', day=day, hours=(4,), xerr=1)
plot_diurnal('P-MD', day=day, hours=(12,), xerr=1)
plot_diurnal('GW', day=day, hours=(8,16), xerr=1)

In [ ]:
day=240
plot_diurnal('P-PD', day=day, hours=(4,), xerr=1)
plot_diurnal('P-MD', day=day, hours=(12,), xerr=1)
plot_diurnal('GW', day=day, hours=(8,16), xerr=1)

In [ ]:
""" From Posch et al.
Mean Tleaf calculated as average of Tleaf measured at 15 min intervals between 14:00 to 18:00
each day in three pseudoreplicate leaves per tree, and three replicate trees per population. (B) The difference between Tleaf and air temperature (Tair) was also
calculated for the same period.
"""
daily_ground_leaft_mean = ground[(ground['standard-time'] >= 14) & (ground['standard-time'] <= 18)].groupby('julian-day')['leaftemp'].mean()
daily_pred_leaft_mean = pred[(pred['standard-time'] >= 14) & (pred['standard-time'] <= 18)].groupby('julian-day')['leaftemp'].mean()
daily_low_leaft_mean = low[(low['standard-time'] >= 14) & (low['standard-time'] <= 18)].groupby('julian-day')['leaftemp'].mean()
daily_high_leaft_mean = high[(high['standard-time'] >= 14) & (high['standard-time'] <= 18)].groupby('julian-day')['leaftemp'].mean()

# Calculate daily mean of (leaftemp - T-air) for ground and predictions, 14:00-18:00
daily_ground_diff_mean = ground[(ground['standard-time'] >= 14) & (ground['standard-time'] <= 18)].groupby('julian-day')['leaftemp'].mean() - pred[(pred['standard-time'] >= 14) & (pred['standard-time'] <= 18)].groupby('julian-day').mean().loc[pred['julian-day'] >= leaft_data_start]['T-air']
daily_pred_diff_mean = pred[(pred['standard-time'] >= 14) & (pred['standard-time'] <= 18)].groupby('julian-day').apply(lambda df: (df['leaftemp'] - df['T-air']).mean(), include_groups=False)
daily_low_diff_mean = low[(low['standard-time'] >= 14) & (low['standard-time'] <= 18)].groupby('julian-day').apply(lambda df: (df['leaftemp'] - df['T-air']).mean(), include_groups=False)
daily_high_diff_mean = high[(high['standard-time'] >= 14) & (high['standard-time'] <= 18)].groupby('julian-day').apply(lambda df: (df['leaftemp'] - df['T-air']).mean(), include_groups=False)

In [ ]:
daily_pred_leaft_count = pred[(pred['standard-time'] >= 14) & (pred['standard-time'] <= 18)].groupby('julian-day')['leaftemp'].count()
daily_pred_leaft_std = pred[(pred['standard-time'] >= 14) & (pred['standard-time'] <= 18)].groupby('julian-day')['leaftemp'].std()
daily_pred_leaft_stderr = daily_pred_leaft_std / np.sqrt(daily_pred_leaft_count)

daily_ground_leaft_count = ground[(ground['standard-time'] >= 14) & (ground['standard-time'] <= 18)].groupby('julian-day')['leaftemp'].count()
daily_ground_leaft_std = ground[(ground['standard-time'] >= 14) & (ground['standard-time'] <= 18)].groupby('julian-day')['leaftemp'].std()
daily_ground_leaft_stderr = daily_ground_leaft_std / np.sqrt(daily_ground_leaft_count)

daily_pred_diff_count = pred[(pred['standard-time'] >= 14) & (pred['standard-time'] <= 18)].groupby('julian-day').apply(lambda df: df['leaftemp'].count(), include_groups=False)
daily_pred_diff_std = pred[(pred['standard-time'] >= 14) & (pred['standard-time'] <= 18)].groupby('julian-day').apply(lambda df: (df['leaftemp'] - df['T-air']).std(), include_groups=False)
daily_pred_diff_stderr = daily_pred_diff_std / np.sqrt(daily_pred_diff_count)

daily_ground_diff_count = ground[(ground['standard-time'] >= 14) & (ground['standard-time'] <= 18)].groupby('julian-day').apply(lambda df: df['leaftemp'].count(), include_groups=False)
daily_ground_diff_std = ground[(ground['standard-time'] >= 14) & (ground['standard-time'] <= 18)].groupby('julian-day').apply(lambda df: (df['leaftemp'] - df['T-air']).std(), include_groups=False)
daily_ground_diff_stderr = daily_ground_diff_std / np.sqrt(daily_ground_diff_count)

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(pred[(pred['standard-time'] >= 14) & (pred['standard-time'] <= 18)].groupby('julian-day').max().loc[pred['julian-day'] >= leaft_data_start]['T-air'])
plt.axvline(x=stress_begin, color='navy', linestyle='--', label='Pre-stress Cutoff')
plt.axvline(x=predrought_cutoff, color='navy', linestyle='--', label='Predrought Cutoff')
plt.axvline(x=post_drought, color='navy', linestyle='--', label='Drought Cutoff')

# Generate date labels from July 24th to September 25th
start_date = datetime(2023, 7, 3)
end_date = datetime(2023, 9, 25)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

julian_days = pred[(pred['standard-time'] >= 14) & (pred['standard-time'] <= 18)].groupby('julian-day').max().loc[pred['julian-day'] >= leaft_data_start]['T-air'].index
tick_indices = []
tick_labels = []
for i, date in enumerate(date_range):
    if i % 7 != 0:
        continue
    jd = date.timetuple().tm_yday
    if jd in julian_days:
        tick_indices.append(jd)
        tick_labels.append(date.strftime('%b %d'))

plt.xticks(tick_indices, tick_labels, rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12,4))

plt.plot(daily_ground_leaft_mean.index, daily_ground_leaft_mean.values, color="navy", alpha=1, label=f'Ground')
plt.fill_between(daily_ground_leaft_stderr.index, daily_ground_leaft_mean.values - 1.96 * daily_ground_leaft_stderr.values, daily_ground_leaft_mean.values + 1.96 * daily_ground_leaft_stderr.values, color='navy', alpha=0.4, label="Ground 95% Interval")

plt.plot(daily_pred_leaft_mean.index, daily_pred_leaft_mean.values, color="green", alpha=1, label=f'Prediction')
plt.fill_between(daily_pred_leaft_stderr.index, daily_pred_leaft_mean.values - 1.96 * daily_pred_leaft_stderr.values, daily_pred_leaft_mean.values + 1.96 * daily_pred_leaft_stderr.values, color='green', alpha=0.4, label="Pred 95% Interval")
# plt.fill_between(daily_high_leaft_mean.index, daily_low_leaft_mean.values, daily_high_leaft_mean.values, color='green', alpha=0.4, label="95% Interval")

plt.axvline(x=stress_begin, color='navy', linestyle='--', label='Pre-stress Cutoff')
plt.axvline(x=predrought_cutoff, color='navy', linestyle='--', label='Predrought Cutoff')
plt.axvline(x=post_drought, color='navy', linestyle='--', label='Drought Cutoff')
plt.xlabel("Timesteps")
plt.ylabel("Leaf temperature C")

# Generate date labels from July 24th to September 25th
start_date = datetime(2023, 7, 24)
end_date = datetime(2023, 9, 25)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Set x-ticks at the corresponding julian-day indices
julian_days = daily_ground_diff_mean.index
tick_indices = []
tick_labels = []
for i, date in enumerate(date_range):
    if i % 7 != 0:
        continue
    jd = date.timetuple().tm_yday
    if jd in julian_days:
        tick_indices.append(jd)
        tick_labels.append(date.strftime('%b %d'))

plt.xticks(tick_indices, tick_labels, rotation=45)
plt.title("Predicted leaf temperature")
plt.legend()

# plt.xlim(start_day, predrought_cutoff)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(daily_ground_diff_mean.index, daily_ground_diff_mean.values, color="navy", alpha=1, label=f'Ground')
plt.fill_between(daily_ground_diff_stderr.index, daily_ground_diff_mean.values - 1.96 * daily_ground_diff_stderr.values, daily_ground_diff_mean.values + 1.96 * daily_ground_diff_stderr.values, color='navy', alpha=0.4, label="Ground 95% Interval")

plt.plot(daily_pred_diff_mean.index, daily_pred_diff_mean.values, color="green", alpha=1, label=f'Prediction')
plt.fill_between(daily_pred_diff_stderr.index, daily_pred_diff_mean.values - 1.96 * daily_pred_diff_stderr.values, daily_pred_diff_mean.values + 1.96 * daily_pred_diff_stderr.values, color='green', alpha=0.4, label="Ground 95% Interval")

# plt.fill_between(daily_high_diff_mean.index, daily_low_diff_mean.values, daily_high_diff_mean.values, color='green', alpha=0.4, label="95% Interval")

plt.axvline(x=stress_begin, color='navy', linestyle='--', label='Pre-stress Cutoff')
plt.axvline(x=predrought_cutoff, color='navy', linestyle='--', label='Predrought Cutoff')
plt.axvline(x=post_drought, color='navy', linestyle='--', label='Drought Cutoff')
plt.axhline(y=0, color='navy', linestyle='--')
plt.xlabel("Timesteps")
plt.ylabel("Leaf temperature - Air temperature C")

# Generate date labels from July 24th to September 25th
start_date = datetime(2023, 7, 24)
end_date = datetime(2023, 9, 25)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Set x-ticks at the corresponding julian-day indices
julian_days = daily_ground_diff_mean.index
tick_indices = []
tick_labels = []
for i, date in enumerate(date_range):
    if i % 7 != 0:
        continue
    jd = date.timetuple().tm_yday
    if jd in julian_days:
        tick_indices.append(jd)
        tick_labels.append(date.strftime('%b %d'))

plt.xticks(tick_indices, tick_labels, rotation=45)
plt.title("Predicted difference in leaf temperature versus air temperature")
plt.legend()

# plt.xlim(start_day, predrought_cutoff)
plt.show()

In [ ]:
res = cmp_pred_to_ground_metrics(['leaftemp'], ground, pred, start_day, predrought_cutoff)

In [ ]:
res

In [ ]:
res = cmp_pred_to_ground_metrics(['leaftemp'], ground, pred, start_day, end_day)

In [ ]:
res